In [101]:
from cmath import rect
import pygame
from time import sleep
pygame.init()
screen = pygame.display.set_mode((1600, 919))

bg = pygame.image.load("big.jpg").convert()
image = pygame.image.load("small.png").convert()

# Viewport (your "canvas region")
viewport_size = (720, 720)
viewport_surface = pygame.Surface(viewport_size)
viewport_rect = pygame.Rect(812, 76, *viewport_size)

zoom_in_rect = pygame.Rect(100, 100, 400, 300)

# Calculate max scale: how much to scale so the rect fills the viewport
# We want the first edge to hit, so use min (not max)
scale_x = viewport_size[0] / zoom_in_rect.width
scale_y = viewport_size[1] / zoom_in_rect.height
max_scale = min(scale_x, scale_y)  # Use min to stop at first edge

In [ ]:
def draw_with_topleft_interpolation(scale_factor):
    """
    Zoom by interpolating from image topleft (0,0) to rect topleft.
    At scale_factor=0: image topleft is at viewport origin
    At scale_factor=1: rect topleft is at viewport origin
    """
    # Interpolate scale from 1.0 to max_scale
    scale = 1 + (max_scale - 1) * scale_factor
    
    # Scale the image
    image_scaled = pygame.transform.smoothscale_by(image, scale)
    
    # Calculate viewport offset using topleft interpolation:
    # image_topleft * (1 - scale_factor) + rect_topleft * scale_factor
    # Since image_topleft is (0, 0), this simplifies to:
    rect_topleft = zoom_in_rect.topleft
    
    offset_x = rect_topleft[0] * scale_factor
    offset_y = rect_topleft[1] * scale_factor
    
    # Position scaled image so that offset point is at viewport origin
    image_pos_x = -offset_x * scale
    image_pos_y = -offset_y * scale

    # Draw the scaled rect at its position on the scaled image
    rect_viewport_x = zoom_in_rect.x * scale + image_pos_x
    rect_viewport_y = zoom_in_rect.y * scale + image_pos_y
    rect_viewport_w = zoom_in_rect.width * scale
    rect_viewport_h = zoom_in_rect.height * scale
    
    # Draw
    viewport_surface.fill((200, 200, 200))
    viewport_surface.blit(image_scaled, (image_pos_x, image_pos_y))
    pygame.draw.rect(viewport_surface, 'red', 
                     (rect_viewport_x, rect_viewport_y, rect_viewport_w, rect_viewport_h), 2)
    
    # Blit to screen
    screen.fill((30, 30, 30))
    screen.blit(bg, (0, 0))
    screen.blit(viewport_surface, viewport_rect)
    refresh()


def draw_with_center_interpolation(scale_factor):
    """
    Zoom by interpolating from image center to rect center.
    At scale_factor=0: image center is at viewport center
    At scale_factor=1: rect center is at viewport center
    """
    # Interpolate scale from 1.0 to max_scale
    scale = 1 + (max_scale - 1) * scale_factor
    
    # Scale the image
    image_scaled = pygame.transform.smoothscale_by(image, scale)
    
    # Get centers in the original image coordinates
    image_center = (image.get_width() / 2, image.get_height() / 2)
    rect_center = zoom_in_rect.center
    
    # Interpolate between image center and rect center
    target_center_x = image_center[0] * (1 - scale_factor) + rect_center[0] * scale_factor
    target_center_y = image_center[1] * (1 - scale_factor) + rect_center[1] * scale_factor
    
    # Calculate where this target center point is on the scaled image
    scaled_target_x = target_center_x * scale
    scaled_target_y = target_center_y * scale
    
    # Position image so this point appears at viewport center
    viewport_center = (viewport_size[0] / 2, viewport_size[1] / 2)
    image_pos_x = viewport_center[0] - scaled_target_x
    image_pos_y = viewport_center[1] - scaled_target_y

    # Draw the scaled rect at its position on the scaled image
    rect_viewport_x = zoom_in_rect.x * scale + image_pos_x
    rect_viewport_y = zoom_in_rect.y * scale + image_pos_y
    rect_viewport_w = zoom_in_rect.width * scale
    rect_viewport_h = zoom_in_rect.height * scale
    
    # Draw
    viewport_surface.fill((200, 200, 200))
    viewport_surface.blit(image_scaled, (image_pos_x, image_pos_y))
    pygame.draw.rect(viewport_surface, 'red', 
                     (rect_viewport_x, rect_viewport_y, rect_viewport_w, rect_viewport_h), 2)
    
    # Blit to screen
    screen.fill((30, 30, 30))
    screen.blit(bg, (0, 0))
    screen.blit(viewport_surface, viewport_rect)
    refresh()

In [121]:
# Test with center interpolation
for scale_factor in [0.0, 0.25, 0.5, 0.75, 1.0]:
    draw_with_center_interpolation(scale_factor)

In [120]:
# Test with center interpolation
for scale_factor in [0.0, 0.25, 0.5, 0.75, 1.0]:
    draw_with_topleft_interpolation(scale_factor)

In [110]:
def refresh():
    pygame.event.pump()  # Process events
    pygame.display.flip()  # Update display
    sleep(1)  # Pause to see the result

In [ ]:
pygame.draw.rect(viewport, outline_color, 
                    (target_rect_img.scale_by), 2)